In [5]:
import tensorflow as tf
import numpy as np 
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt 

from preprocess import *

In [3]:
# 에폭당 정확도와 손실값을 시각화하는 함수
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string],'')
    plt.xlabel('Epochs')
    plt.ylabel('string')
    plt.legend([string, 'val_'+string])
    plt.show()

In [6]:
# 학습 데이터 경로
TRAIN_INPUTS = 'train_inputs.npy' 
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json' 

In [7]:
# 랜덤 시드값
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

In [8]:
# 학습에 필요한 데이터 불러오기
index_inputs = np.load(open(TRAIN_INPUTS, 'rb'))
index_outputs = np.load(open(TRAIN_OUTPUTS, 'rb'))
index_targets = np.load(open(TRAIN_TARGETS, 'rb'))
prepro_configs = json.load(open(DATA_CONFIGS, 'r'))

In [10]:
# 함수를 통과한 값들이 예상한 크기와 같은지 확인
print(len(index_inputs), len(index_outputs), len(index_targets))

11823 11823 11823


In [21]:
MODEL_NAME = 'seq2seq_kor'
BATCH_SIZE = 2
MAX_SEQUENCE = 25
EPOCH = 30
UNITS = 1024 # 재귀 신경망의 결과 차원
EMBEDDING_DIM = 256 # 임베딩 차원 
VALIDATION_SPLIT = 0.1

char2idx = prepro_configs['char2idx'] # 토큰을 인덱스로 만드는 함수
idx2char = prepro_configs['idx2char'] # 인덱스를 토큰으로 변환하는 함수
std_index = prepro_configs['std_symbol'] # 시작 토큰
end_index = prepro_configs['end_symbol'] # 끝 토큰 
vocab_index = prepro_configs['vocab_size'] # 사전의 크기


In [23]:
# 인코더
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        # 임베딩 룩업테이블과 GRU를 구성하기 위한 인자를 입력으로 받는다.
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz #베치 크기
        self.enc_units = enc_units # 재귀 신경망의 결과 차원
        self.vocab_size = vocab_size # 사전 크기
        self.embedding_dim = embedding_dim # 임베딩 차원

        # 사전에 포함된 각 단어를 self.embeddin_dim 차원의 임베딩 벡터로 만드는 함수
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim)
        # GRU 신경망을 만드는 함수
        self.gru = tf.keras.layers.GRU(self.enc_units, # GRU의 결과 차원의 크기 
        return_sequences=True, # 각 시퀀스마다 출력 반환 여부 결정
        return_state=True, # 마지막 상태값 반환 여부
        recurrent_initializer='glorot_uniform') # 초깃값을 무엇으로 할지 선언할 수 있다.(현재는 이전노드와 다음 노드의 개수에 의존하는 방법 사용)

    def call(self, x, hidden): # 입력값 x와 은닉상태 hidden을 받는다
        x = self.embedding(x) # 앞의 embedding함수를 통하여 x를 임베딩 벡터로 만들기
        output, state = self.gru(x, inital_state= hidden) # gru함수에 임베딩 벡터와 재귀 순환망의 초기화 상태로 인자로 받은 은닉 상태값 전달, 결괏값으로 시퀀스의 출력값과 마지막 상태값을 리턴 
        return output, state

    # 배치 크기를 받아 재귀 순환망의 초기에 사용될 크기의 은닉 상태를 만드는 역할
    def initialize_hidden_state(self, inp):
        return tf.zeros(tf.shape(inp)[0], self.enc_units)

In [25]:
# 어텐션
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention,self).__init__()
        self.W1 = tf.keras.layers.Dense(units) 
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)       # units와 1은 출력 크기

    def call(self, query, values):   # query = 인코더 재귀 순환망의 은닉상태값, values = 인코더 재귀 순환망의 결괏값
        hidden_with_time_axis = tf.expand_dims(query, 1)  # query를 행렬곱을 할 수 있는 형태로 변경
        score = self.V(tf.nn.tanh(self.W1(values)+self.W2(hidden_with_time_axis))) # 

        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * values
        context_vector =tf.reduce_sum(context_vector,axis=1) # 차원을 줄이면서 axis 방향으로 더함.

        return context_vector, attention_weights # 어텐션이 계산된 문맥 벡터, 가중치 

In [ ]:
# 디코더
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()

        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim # 임베딩 차원

        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
        return_sequences=True,
        return_state=True,
        recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(self.vocab_size) # 출력 값이 사전 크기인 완전 연결 계층
        
        self.attention = BahdanauAttention(self.dec_units) 
    
    def call(self, x, hidden, enc_output): # 디코더의 입력값 x, 인코더의 은닉 상태 값 hidden, 인코더의 결괏값
        context_vector, attention_weights = self.attention(hidden, enc_output)

        x = self.embedding(x)

        x = tf.concat([tf.expand_dims(context_vector,1),x], axis=-1) # 문맥 벡터와 임베딩 벡터를 결합

        output, state = self.gru(x) # 디코더 순환 신경망을 통과
        output = tf.reshape(output,(-1,output.shape[2])) # 순환 신경망의 결괏값을 얻는다

        x = self.fc(output) # 완전 연결 계층을 통과해서 사전 벡터 크기의 벡터 x를 만든다. 

        return x, state, attention_weights